In [3]:
import os
os.getcwd()

'/mnt/ka_gh_storage/2025_11_27_40x_persister_1HourAmpicillinTreatments'

In [4]:
import zarr
trenches_zarr = zarr.open("trenches.zarr", mode='r')
print(trenches_zarr.shape)
print(trenches_zarr.chunks)

(60420, 321, 1, 135, 24)
(76, 321, 1, 135, 24)


In [5]:
# View trenches
import napari # Visualise masks
viewer = napari.Viewer()
viewer.add_image(trenches_zarr, name='trenches')

<Image layer 'trenches' at 0x72ae921cc750>

In [6]:
import zarr
masks_zarr = zarr.open("masks_upscaled.zarr", mode='r')
print(masks_zarr.shape)
print(masks_zarr.chunks)

(60420, 321, 1, 270, 48)
(1, 64, 1, 270, 48)


In [7]:
# Overlay trenches and masks
import napari
import zarr

# Open your zarr arrays
trenches_zarr = zarr.open("trenches.zarr", mode="r")
masks_zarr = zarr.open("masks_upscaled.zarr", mode='r')

# Create a napari viewer
viewer = napari.Viewer()
viewer.add_image(trenches_zarr, name='trenches', scale=(1, 1, 1, 2, 2)) # 2x upscaled
viewer.add_labels(masks_zarr, name='masks')

napari.run()

In [16]:
# View side-by-side, original vs filtered
import napari
import dask.array as da
import numpy as np

# 1. Open arrays using dask
trenches_dask = da.from_zarr("trenches.zarr")
masks_dask = da.from_zarr("masks_upscaled.zarr")
masks_filtered_dask = da.from_zarr("masks_upscaled_filtered.zarr")
PC_channel = 0

# 2. Slice the Image (Lazy)
# Shape: (T, Z, Y, X)
trenches_channel_1 = trenches_dask[:, :, PC_channel, :, :]

# 3. Handle Masks dimensions (Lazy)
if masks_dask.ndim == 5:
    masks_view = masks_dask[:, :, 0, :, :]
else:
    masks_view = masks_dask

if masks_filtered_dask.ndim == 5:
    masks_view_filtered = masks_filtered_dask[:, :, 0, :, :]
else:
    masks_view_filtered = masks_filtered_dask

# --- CALCULATE GEOMETRY ---

size_x = trenches_channel_1.shape[3]
image_scale = (1, 1, 2, 2)
mask_scale = (1, 1, 1, 1)

display_width = size_x * image_scale[3]
gap = 10 

shift_amount = display_width + gap
translation_vector = (0, 0, 0, shift_amount)

# 4. Create napari viewer
viewer = napari.Viewer()

# --- ORIGINAL ---
viewer.add_image(
    trenches_channel_1, 
    name='Original Image', 
    scale=image_scale,
    blending='additive'
)

viewer.add_labels(
    masks_view, 
    name='Masks (Original)', 
    scale=mask_scale
)

# --- FILTERED ---
viewer.add_image(
    trenches_channel_1, 
    name='Filtered Image', 
    scale=image_scale,
    translate=translation_vector,
    blending='additive'
)

viewer.add_labels(
    masks_view_filtered, 
    name='Masks (Filtered)', 
    scale=mask_scale,
    translate=translation_vector
)

# --- TEXT LABELS ---

# Position: Adjusted to -20 so 'center' alignment doesn't overlap the image top
text_y_pos = -20
center_left = display_width / 2
center_right = display_width + gap + (display_width / 2)

label_points = np.array([
    [text_y_pos, center_left],
    [text_y_pos, center_right]
])

text_properties = {
    'string': ['Original', 'Filtered'],
    'color': 'white',
    'size': 20,     # Fixed screen-pixel size
    'anchor': 'center', # Corrected anchor (valid options: center, upper_left, etc.)
    'translation': np.array([0, 0])
}

viewer.add_points(
    label_points,
    name='Labels',
    size=0,
    text=text_properties,
    face_color='transparent',
    edge_color='transparent'
)

# --- STARTUP SETTINGS ---

viewer.reset_view()

# Force the sliders to the first frame (Time=0, Z=0)
viewer.dims.current_step = (0, 0, 0, 0)

napari.run()

In [15]:
# View side-by-side, original vs new segmentation model/updated code
import napari
import dask.array as da
import numpy as np

# --- 1. LOAD DATA (From Script A) ---
trenches_dask = da.from_zarr("trenches_uncompressed.zarr")
masks_dask = da.from_zarr("masks_upscaled_original.zarr")
masks_dask_2 = da.from_zarr("masks_upscaled.zarr")
PC_chan = 0

# --- 2. SLICE & PREPARE (Lazy) ---
# Shape: (T, Z, Y, X)
trenches_channel_1 = trenches_dask[:, :, PC_chan, :, :]

# Handle Masks dimensions (Lazy)
# Ensure masks are 4D: (T, Z, Y, X)
if masks_dask.ndim == 5:
    masks_view = masks_dask[:, :, 0, :, :]
else:
    masks_view = masks_dask

if masks_dask_2.ndim == 5:
    masks_view_2 = masks_dask_2[:, :, 0, :, :]
else:
    masks_view_2 = masks_dask_2

# --- 3. CALCULATE GEOMETRY (From Script B) ---

# Define scales based on your specific data
image_scale = (1, 1, 2, 2)
mask_scale = (1, 1, 1, 1)

# Get X dimension size (Index 3 in T, Z, Y, X)
size_x = trenches_channel_1.shape[3]

# Calculate display width in "world coordinates" (pixels * scale)
display_width = size_x * image_scale[3]
gap = 10 

# Calculate shift amount
shift_amount = display_width + gap

# Create translation vector for 4D data: (T, Z, Y, X)
# We only want to shift in X (the last dimension)
translation_vector = (0, 0, 0, shift_amount)

# --- 4. NAPARI VIEWER ---
viewer = napari.Viewer()

# === LEFT SIDE (Original) ===
viewer.add_image(
    trenches_channel_1, 
    name='Image (Left)', 
    scale=image_scale,
    blending='additive'
)

viewer.add_labels(
    masks_view, 
    name='Masks (Original)', 
    scale=mask_scale
)

# === RIGHT SIDE (10 Workers) ===
# We add the same image again, but shifted
viewer.add_image(
    trenches_channel_1, 
    name='Image (Right)', 
    scale=image_scale,
    translate=translation_vector,
    blending='additive'
)

viewer.add_labels(
    masks_view_2, 
    name='Masks (New)', 
    scale=mask_scale,
    translate=translation_vector
)

# --- 5. TEXT LABELS ---

# Text Position Calculation
# We place text slightly above the image (Y = -20)
text_y_pos = -20
center_left = display_width / 2
center_right = display_width + gap + (display_width / 2)

# Points are 2D [y, x]. Napari will broadcast these to all T/Z slices.
label_points = np.array([
    [text_y_pos, center_left],
    [text_y_pos, center_right]
])

text_properties = {
    'string': ['Original', 'New'],
    'color': 'white',
    'size': 20,
    'anchor': 'center',
    'translation': np.array([0, 0])
}

viewer.add_points(
    label_points,
    name='Labels',
    size=0,
    text=text_properties,
    face_color='transparent',
    edge_color='transparent'
)

# --- STARTUP SETTINGS ---
viewer.reset_view()
# Force sliders to first frame (Time=0, Z=0)
viewer.dims.current_step = (0, 0, 0, 0)

napari.run()